In [ ]:
#!/usr/bin/env python

from fanuc_pcdk_client import FanucPCDKClient
from robot_loc_world import RobotLocWorld
from robot_loc_joint import RobotLocJoint
from time import sleep
from copy import deepcopy
import time

ROBOT_SPEED = 30
ROBOT_SPEED_LINEAR = 45

In [ ]:
# Robot init
robot = FanucPCDKClient()
robot.connect()

# Run teach pendant program.
robot.run_program('PCDK')
robot.reset_alarms()

# Adjust the speed of the robot
robot.set_speed(ROBOT_SPEED)

In [ ]:
# USE THESE TO GET WORLD POSITIONS AND JOINT POSITIONS

# Retrieve the current world XYZWPR coordinates of the robot
loc_world = robot.get_position("world")
print(f'Current world location: {loc_world}\n')

# Retrieve the current joint coordinates of the robot
loc_joint = robot.get_position("joint")
print(f'Current joint location: {loc_joint}\n')

In [ ]:

# IMPORTANT DATA THAT NEEDS TO BE INPUTTED
# EXPERIMENT A LITTLE BIT, USING JOINT POSITIONS MIGHT MAKE THIS ONE EASIER NOT SURE

VERTICAL_SPACING = 26 # vertical center to center spacing between pegs, change accordingly

### NEEDS TO BE CALIBRATED FOR BOTTOM MOST PEG

left_peg_pose = RobotLocWorld(x=345.437, y=128.544, z=-119.671,
                          w=179.499, p=0.827, r=2.541, 
                          f=False, u=True, t=True)

right_peg_pose = RobotLocWorld(x=345.437, y=128.544, z=-119.671,
                          w=179.499, p=0.827, r=2.541, 
                          f=False, u=True, t=True)

In [ ]:
def generate_trajectories(num_pegs):
    trajectory = []
    
    trajectory.append(home_pose)
    
    left_above = deepcopy(left_peg_pose)
    left_above.z += 5*VERTICAL_SPACING
    
    right_above = deepcopy(right_peg_pose)
    right_above.z += 5*VERTICAL_SPACING
    
    
    for i in reversed(range(1, num_pegs+1)):
        trajectory.append(left_above)
        left = deepcopy(left_peg_pose)
        right = deepcopy(right_peg_pose)
        left.z += (i-1)*VERTICAL_SPACING
        trajectory.append(left)
        trajectory.append('close') # may need to be swapped if on LR Mate 200iD
        trajectory.append(left_above)
        #maybe add home here
        trajectory.append(right_above)
        right.z += (3-i)*VERTICAL_SPACING
        trajectory.append(right)
        trajectory.append('open') # may need to be swapped if on LR Mate 200iD
        trajectory.append(right_above)
        trajectory.append(home_pose)
    
    return trajectory

In [ ]:
requested_pegs = 0
success = 0
def execute_stacking(pegs):
    print("\n\n===============    STARTING PEG STACKING    ===============")
    success = 0
    requested_pegs = pegs
    trajectory = generate_trajectories(pegs)
        
    for pose in traj:
        pegs_moved = len(trajectories[pegs]) - len(success)
        time.sleep(0.5)
        if(pose == 'close'):
            robot.close_gripper()
            print("Picking up the peg #" + str(pegs_moved + 1) + " now!")
        elif(pose == 'open'):
            robot.open_gripper()
            success += 1
            print(" peg #" + str(pegs_moved + 1) + " has been completed!")
        else:
            robot.move(pose, move_linear=True, speed=ROBOT_SPEED_LINEAR)
    
    failed = requested_pegs - success
    
    print("\n\n============================\n      STACKING SUMMARY\n============================")
    print("Successfully stacked " + str((success)) + " pegs")
    print("Failed to stack " + str((failed)) + " pegs")

In [ ]:
terminate = False
user_input = 0
started = False
while not terminate:
    try:
        if(not started):
            print("===============  WELCOME TO PEG STACKING  ===============")
            start = True
            
        user_input = (raw_input('How many pegs are there to move? (press q to quit)\n> '))
        user_input = (user_input) if user_input == 'q' else int(user_input)
        if(user_input == 'q'):
            print("===============    TERMINATING PEG STACKING    ===============")
            terminate = True
        elif(user_input > 0):
            
            '''
            ACTUAL PEG STACK STUFF GOES HERE
            '''
            execute_stacking(user_input)
            
        else:
            print("Invalid choice, please try again.")
            terminate = True
        
        if(available_pegs < 1):
            terminate = True
            
    except ValueError:
        print("Not a valid number, please try again")